In [ ]:
# This is only needed if the notebook is run in VSCode
from IPython.display import clear_output, DisplayHandle
def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

In [ ]:
import sys
sys.path.append('..')
import sklearn
from tsai.basics import *
from swdf.utils import *
my_setup(sklearn)
from matplotlib import dates as mdates
import wandb
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback
from itertools import combinations, chain


In [ ]:
# This is only needed for the MIT supercloud, to fix fastai's LRFinder error 
if torch.cuda.is_available() and torch.cuda.device_count() == 0:
    from fastai.callback.schedule import LRFinder

    @patch_to(LRFinder)
    def after_fit(self):
        self.learn.opt.zero_grad() # Needed before detaching the optimizer for future fits
        tmp_f = self.path/self.model_dir/self.tmp_p/'_tmp.pth'
        if tmp_f.exists():
            self.learn.load(f'{self.tmp_p}/_tmp', with_opt=True, device='cpu')
            self.tmp_d.cleanup()

# Forecasting solar drivers F10, S10, M10 and Y10

Some hints about hyperparameters:
- According to the authors of PatchTST: "The ideal patch length may depend on the dataset, 
but P between {8, 16} seems to be general good numbers."

In [ ]:
config_base = yaml2dict('./config/base.yaml', attrdict=True)
config_solfsmy = yaml2dict('./config/solfsmy.yaml', attrdict=True)
config_solfsmy = config_solfsmy.train
# Merge the two configs (the second one overrides the first one for any keys that are present in both)
config = AttrDict({**config_base, **config_solfsmy})
# Add the architecture config
if config.arch_name.lower() == 'patchtst':
    config.arch = yaml2dict('./config/patchtst.yaml', attrdict=True)
else:
    config.arch = AttrDict()
config

In [ ]:
run = wandb.init(project=config.wandb.project, 
                 config=config,
                 group=config.wandb.group,
                 mode=config.wandb.mode, 
                 anonymous='never') if config.wandb.enabled else None
config = dict2attrdict(run.config) if config.wandb.enabled else config
print(config)

In [ ]:
fname = config.data_path if config.data_url is None else download_data(config.data_url,
                                                                       fname=config.data_path)
fname

In [ ]:
# Read the text file into a pandas DataFrame, ignoring the lines starting with '#'
# Column names: YYYY DDD   JulianDay  F10   F81c  S10   S81c  M10   M81c  Y10   Y81c  Ssrc
df_raw = pd.read_csv(fname, delim_whitespace=True, comment='#', header=None, 
                 names=['Year', 'DDD', 'JulianDay', 'F10', 'F81c', 'S10', 'S81c', 
                        'M10', 'M81c', 'Y10', 'Y81c', 'Ssrc'])
df_raw.head()

F10, S10, M10, and Y10 (81c) have different observation and report times; to standardize reporting, all values are reported in sfu units at 12UT (Universal Time); observations are 3-times daily for F10 (20 UT used), every 5 minutes for S10 (daily average used), twice daily for M10 (7 and 16 UT), and every 1 minute for Y10 (Xrays are each minute while Lya is daily); 

For model inputs the values should be used as a daily value between 0-24 UT for a given calendar date; F10 and S10 are 1-day lagged, M10 is 2-day, and Y10 is 5-day lagged in JB2008; the 81-day centered values are used with the same respective lag times. Ssrc has 4 fields (1 for each index): 

*  0 = (F10, S10, M10, Y10) spline-filled if value or missing if no value; 
* 1 = (F10, M10, Y10) derived or measured index, (S10) SOHO/SEM; 
* 2 = (S10) TIMED/SEE v11; 
* 3 = (S10) SOHO gap (daily); 
* 4 = (S10) SOHO gap (average); 
* 5 = (F10) F10 mean (2 surrounding values), (S10) SDO/EVE; 
* 6 = (S10) GOES/EUVS fill-in, (M10) M10 mean (2 surrounding values); 
* 7 = (S10) S10 scaled to match M10 change from previous day; 
* 8 = (S10) SDO/EVE corrections and all S10 tweaked from sat 12388 delta B%, (Y10) UARS/SOLSTICE V18; 
* 9 = (S10) replace original v4.0h data for versions 4.0 and higher, (Y10) UARS/SOLSTICE v19; 
* A = (S10) TIMED/SEE solar minimum correction; 
* B = (S10) replace with original v4.0h S10 data for versions 4.0 and higher, (M10) SORCE/SOLSTICE/SIM v9; 
* C = (S10) SDO/EVE correction, (Y10) GOES XRS; 
* D = (S10) validated TIMED/SEE, (Y10) GOES XRS and SET composite LYA; 
* E = (S10) S10 composite, (Y10) SET composite LYA; 
* F = (F10, S10, M10, Y10) mean of bordering values

Acronyms:
* SOHO/SEM: Solar and Heliospheric Observatory/ Spacecraft's Solar Extreme-ultraviolet Monitor (SEM)
* SDO/EVE: Solar Dynamics Observatory/Extreme Ultraviolet Variability Experiment.
* UARS/SOLSTICE: Upper Atmosphere Research Satellite/Solar Stellar Irradiance Comparison Experiment
* SORCE/SOLSTICE/SIM: Solar Radiation and Climate Experiment/SOLSTICE/Spectral Irradiance Monitor
* GOES/XRS: Geostationary Operational Environmental Satellite/X-Ray Sensor
* "SET composite LYA" refers to the solar irradiance in the Lyman-alpha (Lyα) wavelength range, as measured by the Solar EUV Experiment Telescope (SET) onboard the Solar Radiation and Climate Experiment (SORCE) spacecraft.

This webpage contains forecasts (paid forecast) that we can use to compare to
https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2020SW002496. It's interesting
to see what they forecast from the previous data in order to try the same thing 
with the neural network

In [ ]:
legend_mapping = {
    '0': '(F10, S10, M10, Y10) spline-filled if value or missing if no value',
    '1': '(F10, M10, Y10) derived or measured index, (S10) SOHO/SEM',
    '2': '(S10) TIMED/SEE v11',
    '3': '(S10) SOHO gap (daily)',
    '4': '(S10) SOHO gap (average)',
    '5': '(F10) F10 mean (2 surrounding values), (S10) SDO/EVE',
    '6': '(S10) GOES/EUVS fill-in, (M10) M10 mean (2 surrounding values)',
    '7': '(S10) S10 scaled to match M10 change from previous day',
    '8': '(S10) SDO/EVE corrections and all S10 tweaked from sat 12388 delta B%, (Y10) UARS/SOLSTICE V18',
    '9': '(S10) replace original v4.0h data for versions 4.0 and higher, (Y10) UARS/SOLSTICE v19',
    'A': '(S10) TIMED/SEE solar minimum correction',
    'B': '(S10) replace with original v4.0h S10 data for versions 4.0 and higher, (M10) SORCE/SOLSTICE/SIM v9',
    'C': '(S10) SDO/EVE correction, (Y10) GOES XRS',
    'D': '(S10) validated TIMED/SEE, (Y10) GOES XRS and SET composite LYA',
    'E': '(S10) S10 composite, (Y10) SET composite LYA',
    'F': '(F10, S10, M10, Y10) mean of bordering values'
}

## Data preprocessing

In [ ]:
# Check if there are any missing values
df_raw.isna().sum()

In [ ]:
# Distinct value of the column Ssrc
df_raw.Ssrc.unique()

In [ ]:
# Separate the Ssrc columns into four colums, one for each character of the string,
# The names of the new columns will be SsrcF10, SsrcS10, SsrcM10, and SsrcY10,
# Cast the new columns into categories. Use a loop
for i, c in enumerate('F10 S10 M10 Y10'.split()):
    df_raw[f'Ssrc_{c}'] = df_raw['Ssrc'].str[i].astype('category')
df_raw[['Ssrc_F10', 'Ssrc_S10', 'Ssrc_M10', 'Ssrc_Y10']].head()

In [ ]:
# See the categories of the column Ssrc_S10
df_raw.Ssrc_S10.cat.categories

In [ ]:
# Convert the JulianDay column to a datetime column, and set it as index
df_raw['Date'] = pd.to_datetime(df_raw['JulianDay'], unit='D', origin='julian')
df_raw['Date'].head()

In [ ]:
# Preparing the values to use in the
S10_legend = [legend_mapping[value] for value in df_raw.Ssrc_S10.cat.categories]  

In [ ]:
# hide

# Plot the variable S10. The color of the line will be determined by the value of Ssrc_S10
fig, ax = plt.subplots(figsize=(20, 7))
scatter = ax.scatter(df_raw.Date, df_raw.S10, c=df_raw.Ssrc_S10.cat.codes, cmap='tab10', s=10)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.set_xlabel('Year')
ax.set_ylabel('S10')
ax.set_title('S10 and Ssrc_S10')

# Legend configuration
ax.set_ylim(0,max(df_raw.S10)+100)
legend = ax.legend(handles=scatter.legend_elements()[0], labels=S10_legend, loc="best", title="Origin")
ax.add_artist(legend)

In [ ]:
# Get the number of values equlas to zero in S10
print((df_raw.S10 == 0).sum())
# convert them to NA
df_raw.loc[df_raw.S10 == 0, 'S10'] = np.nan
print((df_raw.S10 == 0).sum())

In [ ]:
#hide
# plot the variable S10 again
fig, ax = plt.subplots(figsize=(20, 5))
ax.scatter(df_raw.Date, df_raw.S10, c=df_raw.Ssrc_S10.cat.codes, cmap='tab10', s=10)

In [ ]:
datetime_col = 'Date'
freq = '1D'
data_columns_fcst = config.data_columns_fcst
data_columns_time = ['Year', 'DDD']
data_columns = data_columns_fcst + data_columns_time if config.add_time_channels else data_columns_fcst
imputation_method = 'ffill'

# sklearn's preprocessing pipeline
preproc_pipe = sklearn.pipeline.Pipeline([
    ('shrinker', TSShrinkDataFrame()), # shrik dataframe memory usage and set the right dtypes
    ('drop_duplicates', TSDropDuplicates(datetime_col='Date')), # drop duplicates
    ('fill_missing', TSFillMissing(columns=data_columns, method=imputation_method, value=None)), # fill missing data (1st ffill. 2nd value=0)
], verbose=True)

df = preproc_pipe.fit_transform(df_raw)
df

In [ ]:
# In the paper by Licata et al. (2020) (https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2020SW002496),
# authors use a period from October 2012 through the end of 2018 for the benchmarking.
# Therefore, we will set the test set as the same period for our analysis, 
# using the column Date as the timestamp, from October 2012 to the end of 2018. 
# Everything before the test set will be used for training, and everything after the test set
# will be used for validation
test_start_datetime = config.test_start_datetime
test_end_datetime = config.test_end_datetime
valid_start_datetime = config.valid_start_datetime

In [ ]:
# hide 

# Plot the variables F10, S10, M10 and Y10, covering the different periods (training, test and validation)
# with different colors. Do it for the 4 variables mentioned above 
fig, ax = plt.subplots(4, 1, figsize=(20, 10))
 
for i, var in enumerate(['F10', 'S10', 'M10', 'Y10']):
    ax[i].plot(df[var], label='train')
    ax[i].plot(df[var][(df.Date >= test_start_datetime) & (df.Date <= test_end_datetime)],
               label='test')
    ax[i].plot(df[var][(df.Date >= valid_start_datetime)], label='valid')
    ax[i].set_title(var)
    ax[i].legend()
    ax[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) # format x-axis ticks

Ranges extracted from [Licata, Tobiska, y Mehta, «Benchmarking Forecasting Models for Space Weather Drivers»](https://onlinelibrary.wiley.com/doi/abs/10.1029/2020SW002496).

In [ ]:

thresholds = {
    'F10':  [(0,75), (76,150), (151,190), (191, df['F10'].max())],
    'S10': [(0,65), (66,150), (151,215), (216, df['S10'].max())],
    'M10': [(0,72), (73,144), (145,167), (168, df['M10'].max())],
    'Y10': [(0,81), (82,148), (149,165), (166, df['Y10'].max())]
}

In [ ]:
df_cat = get_classified_columns(df_raw, thresholds)
df['Cat'] = df_cat['F10_Cat'].replace(to_replace=get_F10_historical_distribution(thresholds))
df.head()

In [ ]:
test_period = (df_raw.Date >= test_start_datetime) & (df_raw.Date <= test_end_datetime)

df_cat = get_classified_columns(df_raw, thresholds)
df_cat = df_cat[~(test_period)]

historical_distribution = get_F10_historical_distribution(thresholds)

# Function parameter calculation
test_size = df[test_period].shape[0] / df.shape[0]
train_val_size = 1 - test_size
val_size = 0.15 / train_val_size
print(f"Test size: {test_size}, Validation size: {val_size}, Train size: {1 - test_size - 0.15}")

# Best segment size found: 250, val_size: 0.15, test_size: 0.65
best_comb, segments, distribution = find_closest_distribution(df_cat['F10_Cat'], historical_distribution, 250, val_size) 

In [ ]:
best_comb_idxs = [segments[i] for i in best_comb]
validation = df.loc[chain.from_iterable(best_comb_idxs)]

In [ ]:
validation_segments = (df.index.isin(chain.from_iterable(best_comb_idxs)))
train_df = df[~validation_segments & ~test_period] 
train_distribution = get_classified_columns(train_df, thresholds)['F10_Cat'].value_counts(normalize=True).to_dict()

train_distribution

In [ ]:
#|hide
# Plot of the distribution found
plt.figure(figsize=(10, 6))
plt.bar(distribution.keys(), distribution.values(), alpha=0.8, label='Validation Distribution')
plt.bar(historical_distribution.keys(), historical_distribution.values(), alpha=0.5, label='Historical Distribution')
plt.bar(train_distribution.keys(), train_distribution.values(), alpha=0.5, label='Train Distribution')

plt.xlabel('Solar activity categories')
plt.ylabel('Percentage of data')
plt.title('Solar Activity Categories Distribution Comparison of F10.7')
plt.legend()
plt.show()

# Plot of the future splits
fig, ax = plt.subplots(4, 1, figsize=(20, 10))

for i, var in enumerate(['F10', 'S10', 'M10', 'Y10']):
    ax[i].plot(df.Date, df[var], label='train')
    ax[i].plot(df.Date[(df.Date >= test_start_datetime) & (df.Date <= test_end_datetime)],
               df[var][(df.Date >= test_start_datetime) & (df.Date <= test_end_datetime)],
               label='test')
    ax[i].plot(validation.Date,
               validation[var],
               label='validation')
    ax[i].set_title(var)
    ax[i].legend()
    ax[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) # format x-axis ticks
    
    set_ranges = np.zeros((int(df[var].max()), df.shape[0],4)) # (Height, Width)

    for j, (start, end) in enumerate(thresholds[var]):
        set_ranges[int(start):int(end), :, :] = [0,0,0, ((j+1)/4)]
   
    ax[i].imshow(set_ranges, extent=[df.Date.min(), df.Date.max(), 0, df[var].max()], aspect='auto', alpha=0.5, origin='lower')

In [ ]:
import more_itertools as mit

def sliding_window_generator(df, split_start, comb=None, segments=None):
    consecutive_elements, X, y = None, None, None

    if comb is not None:
        consecutive_elements = [list(group) for group in mit.consecutive_groups(comb)]

        df_to_window = []
        for elements in consecutive_elements:
            best_comb_idxs = [segments[i] for i in elements]
            df_to_window.append(df.iloc[chain.from_iterable(best_comb_idxs)])
    else:
        df_to_window = [df]

    X_window, y_window = None, None 
    for df_window in df_to_window:    
        X_window, y_window = SlidingWindow(
            window_len=config.lookback,
            horizon=config.horizon, 
            stride=1, 
            get_x=data_columns, 
            get_y=data_columns
        )(df_window)
        
        X = np.concatenate([X, X_window]) if X is not None else X_window
        y = np.concatenate([y, y_window]) if y is not None else y_window
    
    
    splits = L(list(np.arange(split_start, len(X)+split_start)))
    return X, y, splits

a = np.arange(0,len(segments))
train_comb = list(np.setdiff1d(a, best_comb))

X_val, y_val, split_val = sliding_window_generator(df, 0, comb=best_comb, segments=segments)
X_train, y_train, split_train = sliding_window_generator(df, split_val[-1]+1, comb=train_comb, segments=segments)
X_test, y_test, split_test = sliding_window_generator(df[test_period], split_train[-1]+1) 

X = np.concatenate([X_val, X_train, X_test])
y = np.concatenate([y_val, y_train, y_test])

splits = (split_train, split_val, split_test)
splits, X.shape, y.shape

In [ ]:
# Now that we have defined the splits for this particular experiment, we'll scale
# the data
train_split = splits[0]
exp_pipe = sklearn.pipeline.Pipeline([
    ('scaler', TSStandardScaler(columns=data_columns)),
], verbose=True)
save_object(exp_pipe, 'tmp/exp_pipe.pkl')
exp_pipe = load_object('tmp/exp_pipe.pkl')
# TODO: I don't know why but if I don't copy the dataframe df it gets modified
df_scaled = exp_pipe.fit_transform(df.copy(), scaler__idxs = train_split)
#df_scaled.set_index(datetime_col, inplace=True)
df_scaled.head()

 ### Apply a sliding window. 

### Train

In [ ]:
tsai.__version__


In [ ]:
wandb_callback = WandbCallback(log_preds=False)
cbs = L(wandb_callback) if config.wandb.enabled else L()
learn = TSForecaster(X, y, splits=splits, batch_size=config.bs,
                     pipelines=[preproc_pipe, exp_pipe], arch=config.arch_name, 
                     arch_config=dict(config.arch), 
                     init=config.init_weights,
                     cbs= cbs + ShowGraphCallback(), 
                     partial_n=config.partial_n)
lr_max = learn.lr_find().valley if config.lr_max is None else config.lr_max
print(lr_max)
print(f"#params: {sum(p.numel() for p in learn.model.parameters())}")
learn.fit_one_cycle(n_epoch=config.n_epoch, lr_max=config.lr_max)

In [ ]:
# Print the validation loss and save it in case other notebooks (optuna) wants to
# use it for hyperparameter optimization
valid_loss = learn.validate()[0] 
print(valid_loss)
%store valid_loss

In [ ]:
# Log the test loss to wandb
test_loss = learn.validate(ds_idx=2)[0]
print(test_loss)
if run is not None:
    run.log(dict(test_loss=test_loss))

In [ ]:
# Save everything
learn.dls.loaders += [learn.dls.valid.new_dl(X[splits[2]], y[splits[2]])] # Add test datalaoder
# Remove the wandb callback to avoid errors when downloading the learner
if config.wandb.enabled:
    learn.remove_cb(wandb_callback)

# Save locally and in wandb if online and enabled
learn.save_all(path='tmp', verbose=True) 
if run is not None and config.wandb_mode and config.wandb_log_learner:
    # Save the learner (all tmp/dls, tmp/model.pth, and tmp/learner.pkl). 
    run.log_artifact('tmp', type='learner', name='solfsmy')

In [ ]:
if run is not None:
    run.finish()